## Análsis de la Retroalimentación de Posadas y Gentera GPTW México
## Basado en muestra de datos de Net Promoter Score de ambos hoteles

In [241]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import string
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import concurrent.futures
import time
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import os
print(os.listdir("../input"))

# Plotly based imports for visualization
from plotly import tools
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

# spaCy based imports
import spacy
from spacy.lang.es.stop_words import STOP_WORDS
from spacy.lang.es import Spanish
!python -m spacy download es_core_news_sm

#Pandas display width
pd.set_option('max_colwidth',100) #max width df display

['gptw08', 'gptw07']


You are using pip version 10.0.1, however version 20.3b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

    Linking successful
    /opt/conda/lib/python3.6/site-packages/es_core_news_sm -->
    /opt/conda/lib/python3.6/site-packages/spacy/data/es_core_news_sm

    You can now load the model via spacy.load('es_core_news_sm')



In [242]:
# Data load in csv format
wines = pd.read_csv('../input/gptw08/dbgptwliderplus.csv')

wines.head(5)

,org,id,nps_score,nps_why,algo_especial_emp,algo_cambiar_emp,lider_plus,lider_minus
0,Grupo Posadas,4094711993,7,"Es una buena empresa para trabajar ofrecen buenas prestaciones, pero es muy matado el trabajo; t...",A veces,"Más motivación a personal, si el personal está bien y motivado puede hacer mejor y bien su trabajo",Responsable,Buen humor y es tolerante
1,Grupo Posadas,4038461966,10,Poque me gusta mi trabajo,Porlosebfoquenosfestejan,Porquemegustamuchoelhofel,NaN,Estaré pasillos
2,Grupo Posadas,4022001994,10,"Porque es una buena empresa, hay un ambiente laboral agradable,","Buen ambiente de trabajo, beneficios",Prestaciones,"Entusiasta, honesto, retroalimentación",Organizado
3,Grupo Posadas,4032331988,10,Tiene un aviente muy bueno para trabajar,Hay buena relación en el trabajo en trabajo en ejido,El comedor y el horario del trasporte,Te escucha te da consejos no grita,Puede dar oportunidades de como ayer un trabajo de excelente
4,Grupo Posadas,204361993,10,POR EL BUEN AMBIENTE DE TRABAJO. EN ESTA EMPRESA APRENDES MUCHÍSIMAS COSAS,ES UNA ESCUELA PARA APRENDER MUCHAS COSAS,PERSONAL CAPACITADO,SABER DIRIGIR EL DEPARTAMENTO,HACER MAS PROYECTOS DE TRABAJO


In [243]:
wines.shape

(28409, 8)


## Seccción para hacer los queries correspondientess, se requiere la selección de los valores de los campos de interés. 
## El diseño utiliza "wines" como dataframe principal, utilizo "newdf" como auxiliar.  
##  Validando la extracción correcta de la información, str.len para determinar número mínimo de carcteres

In [244]:
#selecting subsets of the dataframe, note the string length condition
#you have to select socre organization and ALSO the name of the field (column) to extract
newdf=wines[(wines['nps_score']>8)  & (wines['org']=="Gentera") & (wines['lider_minus'].str.len()>20)]
newdf.head(6)

,org,id,nps_score,nps_why,algo_especial_emp,algo_cambiar_emp,lider_plus,lider_minus
15128,Gentera,175811984,10,por que es una empresa donde la persona es el centro y sobre todo nos llevan a vivir los valores...,"el trato, a pesar de los puestos nos tratamos todos se igualmanera, siempre cuidadon las formas ...",los horarios,sencillez,"aprender a escuchar, involucramiento con sus colaboradores"
15129,Gentera,834331992,10,Porq es una empresa muy comprometida con su gente,El trato a la persona,La empresa está muy estructurada,Su conocimiento,Conocer más nuestra persona
15130,Gentera,526311989,9,HAY OPORTUNIDAD DE CRECIMIENTO,NOS GUIA A TRABAJAR CON VALORES\nNOS DA OPORTUNIDAD DE ESTAR CON NUESTRAS FAMILIAS,QUE NOS DIERAN UN MEDIO DE TRASPORTE SEGURO,RESPONSABLE Y HONESTO,COMUNICACION Y TRABAJO EN EQUIPO
15131,Gentera,840831981,10,Prioriza el valor de la persona y la familia,Prioriza el valor de la persona,No,Conoce los procesos,"Manejó del estrés , su forma de tratar a las personas es muy explosiva"
15132,Gentera,908521986,10,Es una buena empresa,El trabajo en equipo,Todo muy bien,Dinámico activo buen ejemplo a seguir,La capacitación constante
15137,Gentera,861841986,10,Es una buena empresa,El ambiente laboral,Ninguno,El conocimiento,Recordar cuando le pides apoyo de un encargo


In [245]:
newdf.shape

(5202, 8)

## Registra el numero de temas a identificar en la siguiente línea

In [246]:
#Aqui se define el numero de temas a buscar y el nombre del campo del data frame que contiene la informacion a analizar
total_topics=4
field_source='lider_minus'

In [247]:
# Reseting the index of the dataframe ****  *** very important resetting data frame index
newdf=newdf.reset_index(drop=True)
newdf.head(5)

,org,id,nps_score,nps_why,algo_especial_emp,algo_cambiar_emp,lider_plus,lider_minus
0,Gentera,175811984,10,por que es una empresa donde la persona es el centro y sobre todo nos llevan a vivir los valores...,"el trato, a pesar de los puestos nos tratamos todos se igualmanera, siempre cuidadon las formas ...",los horarios,sencillez,"aprender a escuchar, involucramiento con sus colaboradores"
1,Gentera,834331992,10,Porq es una empresa muy comprometida con su gente,El trato a la persona,La empresa está muy estructurada,Su conocimiento,Conocer más nuestra persona
2,Gentera,526311989,9,HAY OPORTUNIDAD DE CRECIMIENTO,NOS GUIA A TRABAJAR CON VALORES\nNOS DA OPORTUNIDAD DE ESTAR CON NUESTRAS FAMILIAS,QUE NOS DIERAN UN MEDIO DE TRASPORTE SEGURO,RESPONSABLE Y HONESTO,COMUNICACION Y TRABAJO EN EQUIPO
3,Gentera,840831981,10,Prioriza el valor de la persona y la familia,Prioriza el valor de la persona,No,Conoce los procesos,"Manejó del estrés , su forma de tratar a las personas es muy explosiva"
4,Gentera,908521986,10,Es una buena empresa,El trabajo en equipo,Todo muy bien,Dinámico activo buen ejemplo a seguir,La capacitación constante


In [248]:
newdf.shape

(5202, 8)

In [249]:
# To review element 
newdf[field_source][4]
#print(newdf.count)

'La capacitación constante '

In [250]:
#Updating the wine df with the processed query
wines=newdf
wines.shape

(5202, 8)

In [251]:
#seleccionando campo fuente del dataframe 
wines[field_source][4]

'La capacitación constante '

In [252]:
# Creando un objeto tipo Spacy
# En este caso el objeto es el modelo en Español
nlp = spacy.load('es_core_news_sm')

In [253]:
#Verficando que se esta procesando el registro correcto
doc = nlp(wines[field_source][4])
spacy.displacy.render(doc, style='ent',jupyter=True)

## Econtrando la raíz de los términos (Lemmatization)
## Creación de lista de puntuación y palabras comunes que no añaden mucha informaciónCY


In [254]:
#definicion de puntuacion y STOP WORDS
punctuations = string.punctuation
stopwords = list(STOP_WORDS)
stopwords.append('empresa') # added 'empresa' as a Stop Word

In [255]:
#Quitar signo de comentarios para revisar listas de stopwords y punctuations
#stopwords
#punctuations

In [256]:
review = str(" ".join([i.lemma_ for i in doc]))

In [257]:
doc = nlp(review)
spacy.displacy.render(doc, style='ent',jupyter=True)

## Detectando los componentes dentro de las frases ###



In [258]:
# POS tagging
for i in nlp(review):
    print(i,"=>",i.pos_)

La => DET
capacitación => NOUN
constante => ADJ


In [259]:
# Parser para las frases escritas Añadi len word >4 to filter out short words with no meaning in this exercise
parser = Spanish()
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lower_ for word in mytokens ] #*either this line or the next one
    #mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ] #***this section keeps only the lemma removed
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations and len(word)>4] #Eliminating short words
    mytokens = " ".join([i for i in mytokens])
    return mytokens

In [260]:
# utilizando la variable field_source como la columna a analizar de la base de datos
spacy_tokenizer(wines[field_source][4])

'capacitación constante'

In [261]:
spacy_tokenizer('realmente no lo sé xq sólo se dedica a gritarnos  no tiene manera d decir las cosas sin saltarse')

'realmente dedica gritarnos saltarse'

In [262]:
spacy_tokenizer('pues creo q ninguna porque nos trata como personas inutiles aparte es muy alzado hasta vemos')

'personas inutiles aparte alzado vemos'

In [263]:
# Applies the parser and filters information to get a processed data frame
# Important selection on the column of the wines field_source
tqdm.pandas()
wines["processed_source"] = wines[field_source].progress_apply(spacy_tokenizer)

100%|██████████| 5202/5202 [00:02<00:00, 1737.67it/s]


In [264]:
wines["processed_source"][0:4]

0    aprender escuchar involucramiento colaboradores
1                                            persona
2                                comunicacion equipo
3      manejó estrés forma tratar personas explosiva
Name: processed_source, dtype: object

## Encontrando los temas principales entre todos los documentos

## Creación de un vector de términos


## Aquí es donde se hace el fit a la vectorizacion de la data!

In [265]:
# Creating a vectorizer version without the stop words option
vectorizer = CountVectorizer( lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(wines["processed_source"]) #procesando processed source
#type(data_vectorized)
data_vectorized.shape

(5202, 3985)

In [266]:
NUM_TOPICS = total_topics #parametro desde el principio

In [267]:
# Latent Dirichlet Allocation Model ADDED random_state = 42 to reproduce results
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=20, learning_method='online',verbose=True,random_state=42)
data_lda = lda.fit_transform(data_vectorized)
lda.components_.shape
#type(data_lda)
#print(data_lda.shape)
#type(data_lda)

iteration: 1 of max_iter: 20
iteration: 2 of max_iter: 20
iteration: 3 of max_iter: 20
iteration: 4 of max_iter: 20
iteration: 5 of max_iter: 20
iteration: 6 of max_iter: 20
iteration: 7 of max_iter: 20
iteration: 8 of max_iter: 20
iteration: 9 of max_iter: 20
iteration: 10 of max_iter: 20
iteration: 11 of max_iter: 20
iteration: 12 of max_iter: 20
iteration: 13 of max_iter: 20
iteration: 14 of max_iter: 20
iteration: 15 of max_iter: 20
iteration: 16 of max_iter: 20
iteration: 17 of max_iter: 20
iteration: 18 of max_iter: 20
iteration: 19 of max_iter: 20
iteration: 20 of max_iter: 20


(4, 3985)

In [268]:
# Non-Negative Matrix Factorization Model
nmf = NMF(n_components=NUM_TOPICS)
data_nmf = nmf.fit_transform(data_vectorized) 

In [269]:
# Latent Semantic Indexing Model using Truncated SVD
lsi = TruncatedSVD(n_components=NUM_TOPICS)
data_lsi = lsi.fit_transform(data_vectorized)

In [270]:
# function printing keywords for each topic       *** You may select top terms ***
def selected_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):    #idx and topic become a vector for each iteration through the enumerate command
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i])     #very cleverly algins the names in vectorizer with the topics in model components
                        for i in topic.argsort()[:-top_n - 1:-1]]) # i here is the index orders in descinding mode keep the first 10 value stop value is -11 to 

In [271]:
# Keywords for topics clustered by Latent Dirichlet Allocation
print("LDA Model:")
selected_topics(lda, vectorizer)

LDA Model:
Topic 0:
['falta', 'equipo', 'organizaci', 'comunicaci', 'ocasiones', 'confianza', 'puesto', 'caracter', 'actividades', 'tiempos']
Topic 1:
['escuchar', 'colaboradores', 'trato', 'personas', 'seguimiento', 'der', 'apoyo', 'colaborador', 'cargo', 'hablar']
Topic 2:
['persona', 'personal', 'oportunidad', 'apoyo', 'forma', 'mejorar', 'gente', 'car', 'cter', 'campo']
Topic 3:
['equipo', 'organizacion', 'lider', 'clientes', 'paciencia', 'decisiones', 'manejo', 'conocimiento', 'aprender', 'tomar']


## Habilitar para análsis NMF

In [272]:
# Keywords for topics clustered by Latent Semantic Indexing
#print("NMF Model:")
#selected_topics(nmf, vectorizer)

## Habilitar para análsis LSI

In [273]:
# Keywords for topics clustered by Non-Negative Matrix Factorization
#print("LSI Model:")
#selected_topics(lsi, vectorizer)

## Identificando documentos correspondientes a cada tema

In [274]:
# function printing documents belonging to each topic  *** You may select # of top documents ***
def selected_documents(model,wines2,top_n=30):
    numtemas, tempvar = model.components_.shape
    print("Printing Document belonging to each topic")
    for idx in range(numtemas): #idx and topic become a vector for each iteration through each topic 
        print(" ")
        print("Topic %d:" % (idx))
        print(" ")
        ptop_indices=np.argsort(doc_topic[:,idx])[:-top_n-1:-1] #sorting the indices by the offset column 1 in reverse order, top indices gives the order document
        print(wines2[ptop_indices])
        if (idx==0):
            dfcsvtmp=wines2[ptop_indices].to_frame()  #note the use to frame method as the results were just a series
            dfcsvtmp.insert(1,"orig_topic",idx) #during the first iteration add the column named tema with value idx zero in first iteration
            print(idx)
        else:
            dfcsvtmp2=wines2[ptop_indices].to_frame() #note the use to frame method as the results were just a series
            dfcsvtmp2.insert(1,"orig_topic",idx) # use a tmp dataframe to add the column named tema and fill it with the current idx iteration
            dfcsvtmp=dfcsvtmp.append(dfcsvtmp2) # append to previous data frame including the tema column
    return dfcsvtmp #returning the data frame including all comments and temas
        #print (doc_topic[ptop_indices]) #Uncomment to see probabilities by topic

In [275]:
print(lda.components_.shape)

(4, 3985)


In [276]:
# for each document calculate the probability for each topic in column form *** VERY IMPORTANT ***
doc_topic=lda.transform(data_vectorized) 
doc_probs=pd.DataFrame(doc_topic)
#Top documents by topic
# You can change to the label "processed_source" below to see the parsed sentences in the following line
dfcsv=selected_documents(lda,wines[field_source].str.lower()) #converting results to lower case

Printing Document belonging to each topic
 
Topic 0:
 
611     cambia constantemente de estrategias sin dar un tiempo determinado para probar funcionen, a vece...
240     le falta trabajar en equipo\nle falta empatia\nevitar las prefetencias\nno conoce la formacion p...
295     planificar mejores estrategias.  saber realmente los problemas que hay en cada cuenta el porcque...
367     falta de liderazgo\ntrabajo en equipo\nacuerdos con terceros sin respaldar al.comaborador\nfalta...
335     implementación de nuevas estrategias de venta, cambiar las estrategias para alcanzar los objetiv...
1135    es muy obsesivo con tener todo perfecto, y esto hace que trabajemos con bitácoras tras bitácoras...
541     no se explicarlo porque a veces siento que le da mucha flohera hacer las cosas pero al fibal de ...
166          liderazgo y trabajo en equipo, sabe guiar y orientar, fortalece las actividades mas complejas.
3143    es importante integrar al equipo se pueden realizar actividades que fomen

In [277]:
dfcsv.head()


,lider_minus,orig_topic
611,"cambia constantemente de estrategias sin dar un tiempo determinado para probar funcionen, a vece...",0
240,le falta trabajar en equipo\nle falta empatia\nevitar las prefetencias\nno conoce la formacion p...,0
295,planificar mejores estrategias. saber realmente los problemas que hay en cada cuenta el porcque...,0
367,falta de liderazgo\ntrabajo en equipo\nacuerdos con terceros sin respaldar al.comaborador\nfalta...,0
335,"implementación de nuevas estrategias de venta, cambiar las estrategias para alcanzar los objetiv...",0


In [278]:
doc_probs.iloc[4059,0:4]

0    0.025023
1    0.924933
2    0.025021
3    0.025023
Name: 4059, dtype: float64

In [279]:
doc_probs.head()

,0,1,2,3
0,0.050002,0.450182,0.050002,0.449814
1,0.125001,0.125001,0.624996,0.125001
2,0.743227,0.083334,0.083334,0.090104
3,0.035721,0.036857,0.891662,0.035760
4,0.083342,0.416829,0.083341,0.416489


In [280]:
comments_tab=pd.concat([dfcsv, doc_probs], axis=1, join='inner') #inner joing by index of top n doc with idx and probs ****


In [281]:
comments_tab.head(10)

,lider_minus,orig_topic,0,1,2,3
611,"cambia constantemente de estrategias sin dar un tiempo determinado para probar funcionen, a vece...",0,0.937144,0.020900,0.020868,0.021088
240,le falta trabajar en equipo\nle falta empatia\nevitar las prefetencias\nno conoce la formacion p...,0,0.931551,0.022733,0.022732,0.022984
295,planificar mejores estrategias. saber realmente los problemas que hay en cada cuenta el porcque...,0,0.930024,0.024397,0.022836,0.022744
367,falta de liderazgo\ntrabajo en equipo\nacuerdos con terceros sin respaldar al.comaborador\nfalta...,0,0.924391,0.025014,0.025011,0.025584
335,"implementación de nuevas estrategias de venta, cambiar las estrategias para alcanzar los objetiv...",0,0.924353,0.025331,0.025015,0.025300
1135,"es muy obsesivo con tener todo perfecto, y esto hace que trabajemos con bitácoras tras bitácoras...",0,0.916613,0.027797,0.027794,0.027796
541,no se explicarlo porque a veces siento que le da mucha flohera hacer las cosas pero al fibal de ...,0,0.906168,0.031279,0.031275,0.031277
166,"liderazgo y trabajo en equipo, sabe guiar y orientar, fortalece las actividades mas complejas.",0,0.905243,0.031271,0.031268,0.032218
3143,es importante integrar al equipo se pueden realizar actividades que fomenten la confianza y trab...,0,0.904891,0.031290,0.031256,0.032563
4909,falta de organización en eventos extras del ambiente laboral,0,0.892836,0.035722,0.035721,0.035721


In [282]:
#doc_probs   undocuments to see probabilities

In [283]:
type(wines)

pandas.core.frame.DataFrame

## Seccion de prueba para una frase

In [284]:
# Example of transformation of an individual sentence with its probabilities for each topic
text = spacy_tokenizer("Conoce mucho es perservarnte y simpre predica con el ejemplo muy inteligente")
x = lda.transform(vectorizer.transform([text]))[0]
print('Probabilities by topic (count start with zero): ',x)
print()
print("El tema con mayor probabilidad usando modelo LDA es el tema #",np.argmax(x))

Probabilities by topic (count start with zero):  [0.45364688 0.44625191 0.05004968 0.05005153]

El tema con mayor probabilidad usando modelo LDA es el tema # 0


## Visualizando los Resultado de LDA con la utilería pyLDAvis

In [285]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, data_vectorized, vectorizer, mds='tsne')
dash

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
2      -78.671196 -396.283112       1        1  27.474965
0      -33.778664   -7.443441       2        1  25.272522
3      138.194778 -224.309540       3        1  24.350190
1     -250.644653 -179.417053       4        1  22.902322, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
1777  Default  231.000000          falta  231.000000  30.0000  30.0000
1541  Default  379.000000         equipo  379.000000  29.0000  29.0000
1578  Default  170.000000       escuchar  170.000000  28.0000  28.0000
668   Default  144.000000  colaboradores  144.000000  27.0000  27.0000
2765  Default  106.000000     organizaci  106.000000  26.0000  26.0000
2897  Default  114.000000        persona  114.000000  25.0000  25.0000
768   Default  106.000000     comunicaci  106.000000  24.0000  24.0000
2898  Default  107.000000       personal  107.000000  23.0000  23.0000
2678  Default   89.000000      ocasiones   89.000000  22.0000  22.0000
3869  Default   79.000000          trato   79.000000  21.0000  21.0000
805   Default   83.000000      confianza   83.000000  20.0000  20.0000
2768  Default   80.000000   organizacion   80.000000  19.0000  19.0000
2737  Default  105.000000    oportunidad  105.000000  18.0000  18.0000
1843  Default   84.000000          forma   84.000000  17.0000  17.0000
3176  Default   76.000000         puesto   76.000000  16.0000  16.0000
3511  Default   69.000000    seguimiento   69.000000  15.0000  15.0000
2457  Default   79.000000        mejorar   79.000000  14.0000  14.0000
1909  Default   79.000000          gente   79.000000  13.0000  13.0000
569   Default   72.000000       caracter   72.000000  12.0000  12.0000
1014  Default   76.000000           cter   76.000000  11.0000  11.0000
566   Default   76.000000            car   76.000000  10.0000  10.0000
91    Default   71.000000    actividades   71.000000   9.0000   9.0000
543   Default   74.000000          campo   74.000000   8.0000   8.0000
2314  Default   65.000000          lider   65.000000   7.0000   7.0000
304   Default  128.000000          apoyo  128.000000   6.0000   6.0000
2904  Default  107.000000       personas  107.000000   5.0000   5.0000
3184  Default   69.000000    puntualidad   69.000000   4.0000   4.0000
646   Default   61.000000       clientes   61.000000   3.0000   3.0000
3784  Default   60.000000        tiempos   60.000000   2.0000   2.0000
3262  Default   55.000000           reas   55.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
2900   Topic4   30.548758     personales   31.176244   1.4536  -4.8679
277    Topic4   30.230772       apertura   30.852407   1.4536  -4.8783
2591   Topic4   29.588951    necesidades   30.220070   1.4528  -4.8998
3016   Topic4   28.818608   preferencias   29.442827   1.4525  -4.9262
820    Topic4   28.347390     congruente   28.969886   1.4522  -4.9427
645    Topic4   28.473225        cliente   29.098677   1.4522  -4.9382
870    Topic4   22.899483      constante   23.520639   1.4472  -5.1561
913    Topic4   21.767887      controlar   22.392546   1.4456  -5.2067
1921   Topic4   20.049453          gesti   20.673955   1.4433  -5.2890
446    Topic4   18.907659          ayuda   19.540928   1.4410  -5.3476
3745   Topic4   18.364301   temperamento   18.983497   1.4408  -5.3768
539    Topic4   17.730515        cambios   18.352412   1.4395  -5.4119
2348   Topic4   19.233240          llega   19.910463   1.4393  -5.3305
2558   Topic4   17.061460        motivar   17.690064   1.4378  -5.4504
889    Topic4   16.090897      contestar   16.701743   1.4367  -5.5089
2012   Topic4   15.328974       horarios   15.958850   1.4337  -5.5574
717    Topic4   15.203762      compartir   15.833789   1.4333  -5.5656
3108   Topic4   14.855974       producto   15.485454   1.43

In [286]:
#getting topic coordinates, making name adjustments and adding original topic column
#print(dash.topic_coordinates)
coordinates=pd.DataFrame(dash.topic_coordinates)
coordinates.rename(columns = {'topics':'Topic'}, inplace = True) #changing the field name to Topic
coordinates['Topic'] = 'Topic' + coordinates['Topic'].astype(str) #adding the string "Topic" and then adding the value
coordinates.head(10)
coordinates['orig_topic'] = coordinates.index #create a column based on the original index
coordinates.head(10)
#coordinates.to_csv('coordinates_std.csv')

,x,y,Topic,cluster,Freq,orig_topic
topic,,,,,,
2,-78.671196,-396.283112,Topic1,1,27.474965,2
0,-33.778664,-7.443441,Topic2,1,25.272522,0
3,138.194778,-224.309540,Topic3,1,24.350190,3
1,-250.644653,-179.417053,Topic4,1,22.902322,1


In [287]:
topic_map = coordinates[['orig_topic', 'Topic']]
topic_map.head()

,orig_topic,Topic
topic,,
2,2,Topic1
0,0,Topic2
3,3,Topic3
1,1,Topic4


In [288]:
#converting the topic_map to a dictionary to be used later to rename columns accordingly
#the dictionary has two elements one named orig_topic and the second one named "Topic"
diccionario=topic_map.to_dict()
print(diccionario['Topic'])

{2: 'Topic1', 0: 'Topic2', 3: 'Topic3', 1: 'Topic4'}


In [289]:
#print(dash.topic_info)
topic_info=pd.DataFrame(dash.topic_info)
topic_info.rename(columns = {'Category':'Topic'}, inplace = True) #renaming the category field|
topic_info = topic_info[topic_info['Topic'] != 'Default'] #removing any records with "default" description 
topic_info.head(10)

,Topic,Freq,Term,Total,loglift,logprob
term,,,,,,
2897,Topic1,113.650689,persona,114.282459,1.2864,-3.7361
2898,Topic1,106.751725,personal,107.382346,1.2860,-3.7987
1843,Topic1,84.162680,forma,84.792740,1.2844,-4.0365
1909,Topic1,78.390507,gente,79.016127,1.2839,-4.1075
1014,Topic1,76.035493,cter,76.650908,1.2838,-4.1380
566,Topic1,76.035493,car,76.650908,1.2838,-4.1380
2457,Topic1,78.743070,mejorar,79.382475,1.2838,-4.1030
543,Topic1,73.806420,campo,74.429781,1.2835,-4.1678
3184,Topic1,69.339913,puntualidad,69.964438,1.2829,-4.2302


In [290]:
#merging original topic mapping with new mapping for topic_info
merged_topic_info = pd.merge(topic_map,topic_info,on="Topic",how='inner')
merged_topic_info.head(10)
#merged_topic_info.to_csv('topic_info_std.csv')

,orig_topic,Topic,Freq,Term,Total,loglift,logprob
0,2,Topic1,113.650689,persona,114.282459,1.2864,-3.7361
1,2,Topic1,106.751725,personal,107.382346,1.2860,-3.7987
2,2,Topic1,84.162680,forma,84.792740,1.2844,-4.0365
3,2,Topic1,78.390507,gente,79.016127,1.2839,-4.1075
4,2,Topic1,76.035493,cter,76.650908,1.2838,-4.1380
5,2,Topic1,76.035493,car,76.650908,1.2838,-4.1380
6,2,Topic1,78.743070,mejorar,79.382475,1.2838,-4.1030
7,2,Topic1,73.806420,campo,74.429781,1.2835,-4.1678
8,2,Topic1,69.339913,puntualidad,69.964438,1.2829,-4.2302
9,2,Topic1,54.210871,acompa,54.824744,1.2806,-4.4763


In [291]:
merged_comments= pd.merge(topic_map,comments_tab,on="orig_topic",how='inner')
merged_comments.head(10)

,orig_topic,Topic,lider_minus,0,1,2,3
0,2,Topic1,"ella decide quien crece y quien no, no nos da oportunidad de crecer y ciente tener el derecho de...",0.016755,0.016860,0.949468,0.016917
1,2,Topic1,ninguna por por mas advertencias que le den siguecon sus mismos malos modos y formas pesimas d...,0.019241,0.019387,0.942130,0.019242
2,2,Topic1,no acepta ir a lugares lejanos no acompaña de forma regular para verificar la función de los gru...,0.019287,0.019478,0.941849,0.019387
3,2,Topic1,"estar mas al pendiente del área de sucursales, gestionar bien, que sucursales necesitan persona...",0.025008,0.025009,0.924875,0.025109
4,2,Topic1,veo oportunidad para sincronizar el apoyo de las areas de servicio para hacerlas de manera homog...,0.025010,0.025229,0.924677,0.025084
5,2,Topic1,"no sólo mantener - trabajar la agenda que desde dirección o desde él se genera, sino buscar alia...",0.025424,0.025080,0.924484,0.025012
6,2,Topic1,"que quizás es permisible, cosa que con gente adulta, no hay problema, sin embargo hay gente con ...",0.027786,0.027787,0.916627,0.027801
7,2,Topic1,modos y formas \nte corrige utilizando palabras qué te evidencian en lugar que te inspiren hacer...,0.027797,0.027998,0.916407,0.027798
8,2,Topic1,"decirnos en que necesitamos mejorar, ayudarnos a nosotros también a trabajar en áreas de oportun...",0.028056,0.027787,0.916356,0.027801
9,2,Topic1,"que necesita sonreir un poco mas, su aspecto pareciera que esta siempre enojada jijij aunque se ...",0.031273,0.031274,0.906180,0.031273


In [292]:
merged_comments.rename(columns=diccionario['Topic'], inplace=True) #using the dictonary to rename columns using topic_map above

In [293]:
print(merged_comments.columns)

Index(['orig_topic', 'Topic', 'lider_minus', 'Topic2', 'Topic4', 'Topic1',
       'Topic3'],
      dtype='object')


In [294]:
merged_comments.head(10)

,orig_topic,Topic,lider_minus,Topic2,Topic4,Topic1,Topic3
0,2,Topic1,"ella decide quien crece y quien no, no nos da oportunidad de crecer y ciente tener el derecho de...",0.016755,0.016860,0.949468,0.016917
1,2,Topic1,ninguna por por mas advertencias que le den siguecon sus mismos malos modos y formas pesimas d...,0.019241,0.019387,0.942130,0.019242
2,2,Topic1,no acepta ir a lugares lejanos no acompaña de forma regular para verificar la función de los gru...,0.019287,0.019478,0.941849,0.019387
3,2,Topic1,"estar mas al pendiente del área de sucursales, gestionar bien, que sucursales necesitan persona...",0.025008,0.025009,0.924875,0.025109
4,2,Topic1,veo oportunidad para sincronizar el apoyo de las areas de servicio para hacerlas de manera homog...,0.025010,0.025229,0.924677,0.025084
5,2,Topic1,"no sólo mantener - trabajar la agenda que desde dirección o desde él se genera, sino buscar alia...",0.025424,0.025080,0.924484,0.025012
6,2,Topic1,"que quizás es permisible, cosa que con gente adulta, no hay problema, sin embargo hay gente con ...",0.027786,0.027787,0.916627,0.027801
7,2,Topic1,modos y formas \nte corrige utilizando palabras qué te evidencian en lugar que te inspiren hacer...,0.027797,0.027998,0.916407,0.027798
8,2,Topic1,"decirnos en que necesitamos mejorar, ayudarnos a nosotros también a trabajar en áreas de oportun...",0.028056,0.027787,0.916356,0.027801
9,2,Topic1,"que necesita sonreir un poco mas, su aspecto pareciera que esta siempre enojada jijij aunque se ...",0.031273,0.031274,0.906180,0.031273


In [295]:
merged_comments.to_csv('topicdb_std.csv')

In [296]:
#print(dash.token_table)
tok_table=pd.DataFrame(dash.token_table)
tok_table.head(10)
tok_table.to_csv('tok_table2.csv')

In [297]:
#pyLDAvis.enable_notebook()
#dash = pyLDAvis.sklearn.prepare(nmf, data_vectorized, vectorizer, mds='tsne')
#dash

## Cómo interpretar ésta gráfica?
1. Los temas están a la izquierda y sus palabras respectivas a la derecha.
2. Los temas más grandes con más frecuents y entre más cercanos más parecidos son.
3. La selección de las palabras está basada en su capacidad de diferenciación y frecuencia.

**Selecciona el tema para ver sus palabras correspondientes.**

## Biagram spaCy tokenizer para la identificación de temas

In [298]:
def spacy_bigram_tokenizer(phrase):
    doc = parser(phrase) # create spacy object
    token_not_noun = []
    notnoun_noun_list = []
    noun = ""

    for item in doc:
        if item.pos_ != "NOUN": # separate nouns and not nouns
            token_not_noun.append(item.text)
        if item.pos_ == "NOUN":
            noun = item.text
        
        for notnoun in token_not_noun:
            notnoun_noun_list.append(notnoun + " " + noun)

    return " ".join([i for i in notnoun_noun_list])

In [299]:
bivectorizer = CountVectorizer(min_df=5, max_df=0.9, lowercase=True, ngram_range=(1,2))
bigram_vectorized = bivectorizer.fit_transform(wines["processed_source"])

## LDA para información procesada con Biagram

In [300]:
bi_lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online',verbose=True)
data_bi_lda = bi_lda.fit_transform(bigram_vectorized)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


### Temas pare el modelo de bigram 

In [301]:
print("Bi-LDA Model:")
selected_topics(bi_lda, bivectorizer)

Bi-LDA Model:
Topic 0:
['escuchar', 'colaboradores', 'personas', 'caracter', 'forma', 'trato', 'decisiones', 'clientes', 'laboral', 'humildad']
Topic 1:
['equipo', 'falta', 'comunicación', 'organización', 'apoyo', 'ocasiones', 'organizacion', 'mejorar', 'confianza', 'puesto']
Topic 2:
['persona', 'lider', 'oportunidad', 'acompañamiento', 'falta', 'aprender', 'situaciones', 'colaboradores', 'reconocer', 'areas']
Topic 3:
['gente', 'campo', 'seguimiento', 'personal', 'áreas', 'crecimiento', 'paciencia', 'carácter', 'comunicacion', 'cargo']


In [302]:
bi_dash = pyLDAvis.sklearn.prepare(bi_lda, bigram_vectorized, bivectorizer, mds='tsne')
bi_dash

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
1      351.755249 -279.098785       1        1  29.378274
0       77.331848   25.298946       2        1  25.756346
3     -227.021927 -249.173996       3        1  23.174422
2       47.402157 -553.571777       4        1  21.690959, topic_info=     Category        Freq   ...     loglift  logprob
term                        ...                     
265   Default  429.000000   ...     30.0000  30.0000
279   Default  198.000000   ...     29.0000  29.0000
541   Default  138.000000   ...     28.0000  28.0000
132   Default  119.000000   ...     27.0000  27.0000
518   Default  118.000000   ...     26.0000  26.0000
115   Default  168.000000   ...     25.0000  25.0000
89    Default   88.000000   ...     24.0000  24.0000
432   Default   81.000000   ...     23.0000  23.0000
360   Default   94.000000   ...     22.0000  22.0000
497   Default  100.000000   ...     21.0000  21.0000
644   Default   82.000000   ...     20.0000  20.0000
516   Default   91.000000   ...     19.0000  19.0000
96    Default   81.000000   ...     18.0000  18.0000
459   Default   88.000000   ...     17.0000  17.0000
510   Default  125.000000   ...     16.0000  16.0000
590   Default   86.000000   ...     15.0000  15.0000
43    Default  145.000000   ...     14.0000  14.0000
199   Default   74.000000   ...     13.0000  13.0000
320   Default  264.000000   ...     12.0000  12.0000
545   Default  123.000000   ...     11.0000  11.0000
109   Default   72.000000   ...     10.0000  10.0000
18    Default   80.000000   ...      9.0000   9.0000
718   Default   64.000000   ...      8.0000   8.0000
597   Default   77.000000   ...      7.0000   7.0000
277   Default   76.000000   ...      6.0000   6.0000
13    Default   59.000000   ...      5.0000   5.0000
542   Default  123.000000   ...      4.0000   4.0000
137   Default   93.000000   ...      3.0000   3.0000
178   Default   60.000000   ...      2.0000   2.0000
527   Default   58.000000   ...      1.0000   1.0000
...       ...         ...   ...         ...      ...
50     Topic4   49.381280   ...      1.5137  -4.0475
662    Topic4   46.988782   ...      1.5129  -4.0971
53     Topic4   42.271987   ...      1.5115  -4.2029
610    Topic4   42.359053   ...      1.5114  -4.2009
227    Topic4   40.552744   ...      1.5105  -4.2444
500    Topic4   39.147856   ...      1.5101  -4.2797
544    Topic4   36.824570   ...      1.5088  -4.3409
578    Topic4   36.644329   ...      1.5087  -4.3458
483    Topic4   36.130778   ...      1.5083  -4.3599
560    Topic4   34.896377   ...      1.5075  -4.3947
147    Topic4   34.250206   ...      1.5074  -4.4133
535    Topic4   34.694675   ...      1.5073  -4.4005
158    Topic4   27.821410   ...      1.5027  -4.6212
245    Topic4   27.866357   ...      1.5023  -4.6196
420    Topic4   27.271820   ...      1.5022  -4.6412
122    Topic4   25.538793   ...      1.5007  -4.7068
528    Topic4   23.173848   ...      1.4977  -4.8040
54     Topic4   22.518300   ...      1.4973  -4.8327
437    Topic4   23.109503   ...      1.4972  -4.8068
69     Topic4   22.424442   ...      1.4964  -4.8369
141    Topic4   21.971821   ...      1.4959  -4.8573
88     Topic4   21.887291   ...      1.4959  -4.8611
300    Topic4   21.665019   ...      1.4957  -4.8713
636    Topic4   21.316934   ...      1.4952  -4.8875
631    Topic4   21.338382   ...      1.4951  -4.8865
260    Topic4   22.004868   ...      1.4948  -4.8558
191    Topic4   21.203501   ...      1.4944  -4.8929
510    Topic4   74.160526   ...      1.0009  -3.6408
115    Topic4   44.304183   ...      0.1933  -4.1560
320    Topic4   51.605539   ...     -0.1041  -4.0034

[174 rows x 6 columns], token_table=      Topic      Freq               Term
term                                    
10        1  0.985988       acercamiento
13        4  0.995816     acompañamiento
15        3  0.991534            actitud
18        1  0.986347       

In [303]:
import pandas as pd
master_list = [['cat', 123, 'yellow'], ['dog', 12345, 'green'], ['horse', 123456, 'red']]
df = pd.DataFrame(master_list)

df

,0,1,2
0,cat,123,yellow
1,dog,12345,green
2,horse,123456,red


In [304]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.colheader_justify','light', 'display.width', 2000, 'display.max_colwidth', 500):
    df = df.stack().str.lstrip().unstack()
    df = df.style.set_properties(**{'text-align': 'left'})
df

,0,1,2
0,cat,nan,yellow
1,dog,nan,green
2,horse,nan,red


In [305]:
from collections import Counter
topics = lda.show_topics(formatted=False)
data_flat = [w for w_list in data_ready for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])        


AttributeError: 'LatentDirichletAllocation' object has no attribute 'show_topics'

In [ ]:
doc_topic


In [ ]:
dir(LatentDirichletAllocation)

In [ ]:
dir(selected_topics)

In [ ]:
dir(lda)

In [ ]:
selected_topics